In [2]:
%run "nb_spatial_analysis_functions"

StatementMeta(, c3eba054-5e1e-4a41-a596-c1c49ffac253, 14, Finished, Available, Finished)

In [3]:
# Coordinates and month
lat = 52.629729
lng = -1.131592
date = '2024-01'

# API URL
url = f'https://data.police.uk/api/crimes-street/all-crime?lat={lat}&lng={lng}&date={date}'

# Fetch data
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Define nested schema
schema = StructType([
    StructField("category", StringType(), True),
    StructField("persistent_id", StringType(), True),
    StructField("location_subtype", StringType(), True),
    StructField("id", StringType(), True),
    StructField("location_type", StringType(), True),
    StructField("context", StringType(), True),
    StructField("month", StringType(), True),
    StructField("location", StructType([
        StructField("latitude", StringType(), True),
        StructField("longitude", StringType(), True),
        StructField("street", StructType([
            StructField("id", StringType(), True),
            StructField("name", StringType(), True)
        ]))
    ])),
    StructField("outcome_status", StructType([
        StructField("category", StringType(), True),
        StructField("date", StringType(), True)
    ]))
])

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Select nested fields and flatten for display
df_flat = df.select(
    "category",
    "month",
    "location.latitude",
    "location.longitude",
    "location.street.name",
    col("outcome_status.category").alias("outcome_category"),
    col("outcome_status.date").alias("outcome_date")
)

df_flat.createOrReplaceTempView("crimes")
spark.sql("SELECT category, COUNT(*) AS count FROM crimes GROUP BY category").show()

StatementMeta(, c3eba054-5e1e-4a41-a596-c1c49ffac253, 15, Finished, Available, Finished)

+--------------------+-----+
|            category|count|
+--------------------+-----+
|anti-social-behav...|  124|
|            burglary|   60|
|       bicycle-theft|   39|
|criminal-damage-a...|  118|
|               drugs|   72|
|possession-of-wea...|   18|
|         other-theft|  132|
|        public-order|  176|
|         shoplifting|  110|
|       vehicle-crime|   63|
|theft-from-the-pe...|   32|
|             robbery|   22|
|       violent-crime|  486|
|         other-crime|   20|
+--------------------+-----+

